In [1]:
import os 
from typing import Annotated
from openai import AsyncOpenAI
import pandas as pd
from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

In [2]:
# Cargar el archivo CSV
df = pd.read_csv('Database/BloodPressuredataset.csv')

# Mostrar las primeras 5 filas
print("Primeras filas del archivo:")
print(df.head())

# Mostrar información general de las columnas
print("\nInformación de las columnas:")
print(df.info())

# Estadísticas básicas de las columnas numéricas
print("\nEstadísticas descriptivas:")
print(df.describe())

Primeras filas del archivo:
   Edad     Género   IMC  Glucosa  Colesterol  Frecuencia_Cardiaca  \
0    56   Femenino  16.3    104.4       203.7                   86   
1    69  Masculino  20.8    113.4       192.9                   93   
2    46   Femenino  25.7     90.8       147.1                   80   
3    32   Femenino  26.3     95.3       159.5                   89   
4    60  Masculino  28.0     77.8       206.6                   92   

  Nivel_Estres  Presion_Sistolica  Presion_Diastolica  
0         Bajo              143.9                95.5  
1        Medio              113.5                70.2  
2         Alto              109.0                67.6  
3         Bajo              111.3                71.8  
4         Bajo              118.0                99.5  

Información de las columnas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               -------

In [3]:
import pandas as pd
from typing import Annotated


class HealthDataPlugin:
    """Proporciona funciones basadas en datos médicos de pacientes."""

    def __init__(self, csv_path: str):
        # Cargar el dataset de pacientes desde un archivo CSV
        self.df = pd.read_csv(csv_path)

    @kernel_function(description="Devuelve el promedio de presión sistólica de todos los pacientes.")
    def get_avg_systolic_pressure(self) -> Annotated[float, "Promedio de presión sistólica"]:
        return float(self.df["Presion_Sistolica"].mean())

    @kernel_function(description="Devuelve la cantidad de pacientes con IMC mayor a 30 (obesidad).")
    def get_obese_patient_count(self) -> Annotated[int, "Cantidad de pacientes con IMC alto"]:
        return int((self.df["IMC"] > 30).sum())

    @kernel_function(description="Devuelve la edad promedio de los pacientes.")
    def get_avg_age(self) -> Annotated[float, "Edad promedio de los pacientes"]:
        return float(self.df["Edad"].mean())

In [4]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

In [5]:
health_plugin = HealthDataPlugin("Database/BloodPressuredataset.csv")

agent = ChatCompletionAgent(
    service=chat_completion_service,
    plugins=[health_plugin],
    name="VitalMind",
    instructions=(
        "Eres VitalMind, un agente experto en salud. "
        "Puedes acceder a información médica real de pacientes desde un archivo CSV. "
        "Responde de manera clara, empática y útil a las preguntas del usuario."
    )
)

In [8]:
async def main():
# Crear un nuevo hilo para el agente
# Si no se proporciona ningún hilo, 
# se creará uno nuevo y se devolverá con la respuesta inicial.
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Tengo 75 años y me han subido los niveles de glucosa, ¿qué puedo hacer?",
        "Mi padre tiene 82 años y a veces se le olvida tomar sus medicamentos, ¿cómo puedo ayudarlo?",
        "¿Qué debo hacer si tengo presión arterial alta y me siento mareado?",
    ]
    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()

# User: Tengo 75 años y me han subido los niveles de glucosa, ¿qué puedo hacer?

# VitalMind: Es comprensible que te sientas preocupado por el aumento de tus niveles de glucosa, especialmente a los 75 años. Aquí hay algunas recomendaciones útiles:

1. **Consulta a un médico**: Es importante que hables con tu médico para obtener una evaluación completa de tu salud y recibir tratamiento adecuado si es necesario.

2. **Alimentación saludable**: Mantener una dieta equilibrada y rica en nutrientes puede ayudar a controlar los niveles de glucosa. Considera incorporar más frutas, verduras, granos enteros y proteínas magras. Intenta reducir el consumo de azúcares añadidos y carbohidratos refinados.

3. **Ejercicio regular**: Actividad física regular, como caminar, nadar o hacer ejercicio suave, puede mejorar la sensibilidad a la insulina y ayudar a controlar los niveles de glucosa.

4. **Control de peso**: Si tienes sobrepeso, perder incluso una pequeña cantidad de peso puede ayudar a mejorar 